In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import gc

os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/ID_Migration/By_Division'

In [2]:
product_taxonomy=pd.read_table("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20190701-134908-900.txt",dtype=str,sep="|")
product_taxonomy=product_taxonomy[['division_id','class_code_id','subclass_id']].drop_duplicates()
division_table=product_taxonomy.copy()

divison_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Division Names.txt",dtype=str,sep="|")
product_taxonomy=pd.merge(product_taxonomy,divison_name,on="division_id",how="left")

In [3]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root, file)

In [4]:
excel_sotf=pd.ExcelFile("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/ID_Migration/By_Division/Master Zip Doc 2018 to Q1 - Updated 9.3 V2.xlsx")
print(excel_sotf.sheet_names)

SOTF_df=excel_sotf.parse("Store ID List",dtype=str,header=None,usecols=[0])

SOTF_df.columns=['location_id']
SOTF_df['store_type']="SOTF"
SOTF_df=SOTF_df.append(pd.DataFrame({"location_id":"6990","store_type":"Ecommerce"},index=[0]))
SOTF_df=SOTF_df.reset_index()
del SOTF_df['index']

['Store ID List', '7.27 48 Stores', '8.31 6 Stores', '9.14 3 Stores', '10.05 42 Stores', '10.26  29 Stores', '11.09 9 Stores', '11.30 9 Stores', '12.14 7 Stores', '2.8 2019 - 1 Store', '3.1 2019 - 2 Stores', '3.29 2019 - 3 Stores', '4.26 2019 - 5 Stores']


In [5]:
# remove the recent Q1 opened SOTF

remove_tabs=excel_sotf.sheet_names[-4:]
print(remove_tabs)

for tab in remove_tabs:
    try:
        df=excel_sotf.parse(tab,usecols=["location_id"],dtype=str)
        SOTF_df=SOTF_df[~SOTF_df['location_id'].isin(df['location_id'].unique().tolist())]
    except:
        df=excel_sotf.parse(tab,usecols=["Store ID"],dtype=str)
        SOTF_df=SOTF_df[~SOTF_df['location_id'].isin(df['Store ID'].unique().tolist())]
    print(SOTF_df.shape)



['2.8 2019 - 1 Store', '3.1 2019 - 2 Stores', '3.29 2019 - 3 Stores', '4.26 2019 - 5 Stores']
(201, 2)
(199, 2)
(196, 2)
(191, 2)


In [6]:
SOTF_df.tail(2)

,location_id,store_type
189,4596,SOTF
201,6990,Ecommerce


In [7]:
file_list_daily_sales_2019Q1=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
file_list_daily_sales_2019Q1=[x for x in file_list_daily_sales_2019Q1 if "2019-02-09" not in x]
file_list_daily_sales_2019Q1=sorted([x for x in file_list_daily_sales_2019Q1 if "aily" in x])


df_files_2019Q1_daily=pd.DataFrame({"file_path":file_list_daily_sales_2019Q1})
df_files_2019Q1_daily['week_end_dt']=df_files_2019Q1_daily['file_path'].apply(lambda x: x.split("ks/MediaStorm_")[1][:10])

df_files_2019Q1_daily=df_files_2019Q1_daily[((df_files_2019Q1_daily['week_end_dt']>="2019-02-03") & (df_files_2019Q1_daily['week_end_dt']<="2019-05-04"))]
df_files_2019Q1_daily.shape


(12, 2)

In [8]:
df=pd.DataFrame({"file_path":"/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20190209.txt","week_end_dt":"2019-02-09"},index=[0])

df_files_2019Q1_daily=df_files_2019Q1_daily.append(df)
df_files_2019Q1_daily=df_files_2019Q1_daily.sort_values("week_end_dt")
df_files_2019Q1_daily=df_files_2019Q1_daily.reset_index()
del df_files_2019Q1_daily['index']
df_files_2019Q1_daily.shape

(13, 2)

In [9]:
samplerows=None
df_2019Q1_sales_all=pd.DataFrame()
df_2019Q1_sales_div=pd.DataFrame()
i_counter=0
for file in df_files_2019Q1_daily['file_path'].tolist():
    df=pd.read_table(file,sep="|",dtype=str,nrows=samplerows,usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','class_code_id','subclass_id','item_transaction_units','item_transaction_amt'])
    df['rewards_label']=np.where(pd.isnull(df['customer_id_hashed']),"Non_Rewards","Rewards")
    df=pd.merge(df,SOTF_df,on="location_id",how="left")
    df['store_type']=df['store_type'].fillna("Legacy")
    
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df['item_transaction_units']=df['item_transaction_units'].astype(int)
    # Negative sales included in the transction count
    
    df_trans_all=df[['location_id','store_type','transaction_dt','transaction_id','customer_id_hashed','rewards_label']].drop_duplicates()
    df_trans_all=df_trans_all.groupby(['location_id','store_type','transaction_dt','rewards_label'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans"})
    
    df_sales_all=df.groupby(['location_id','store_type','transaction_dt','rewards_label'])['item_transaction_amt','item_transaction_units'].sum().reset_index().rename(columns={"item_transaction_amt":"sales","item_transaction_units":"units"})
    df_all=pd.merge(df_sales_all,df_trans_all,on=['location_id','store_type','transaction_dt','rewards_label'])
    
    
    # division level
    df=pd.merge(df,division_table,on=['class_code_id','subclass_id'],how="left")
    df_trans_div=df[['location_id','store_type','transaction_dt','transaction_id','customer_id_hashed','division_id','rewards_label']].drop_duplicates()
    df_trans_div=df_trans_div.groupby(['location_id','store_type','transaction_dt','rewards_label','division_id'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans"})
    
    df_sales_div=df.groupby(['location_id','store_type','transaction_dt','rewards_label','division_id'])['item_transaction_amt','item_transaction_units'].sum().reset_index().rename(columns={"item_transaction_amt":"sales",'item_transaction_units':'units'})
    df_div=pd.merge(df_sales_div,df_trans_div,on=['location_id','store_type','transaction_dt','rewards_label','division_id'])
    
    df_2019Q1_sales_div=df_2019Q1_sales_div.append(df_div)
    i_counter+=1
    print(datetime.datetime.now(),i_counter,file)
    
del df
del df_all
del df_div
gc.collect()

2019-09-04 11:48:19.539151 1 /home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20190209.txt
2019-09-04 11:53:35.879752 2 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-02-16/MediaStormDailySales20190219-113605-481.txt
2019-09-04 11:57:23.232652 3 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-02-23/MediaStormDailySales20190226-112611-940.txt
2019-09-04 12:07:42.250953 4 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-02/MediaStormDailySales20190305-112738-899.txt
2019-09-04 12:12:00.478888 5 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-09/MediaStormDailySales20190312-121205-996.txt
2019-09-04 12:15:04.585239 6 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-16/MediaStormDailySales20190319-112334-057.txt
2019-09-04 12:25:27.511936 7 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-23/MediaStormDailySales20190326-112740-903.txt
2019-09-04 12:27:11.545589 8 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-30/MediaStormDailySales2

625

In [10]:
df_2019Q1_sales_div.head(2)

,location_id,store_type,transaction_dt,rewards_label,division_id,sales,units,trans
0,1,SOTF,2019-02-03,Non_Rewards,1,966.09,605,105
1,1,SOTF,2019-02-03,Non_Rewards,10,3.00,2,1


In [11]:
df_sales_all.head(2)

,location_id,store_type,transaction_dt,rewards_label,sales,units
0,1,SOTF,2019-04-28,Non_Rewards,5485.17,1179
1,1,SOTF,2019-04-28,Rewards,9644.96,1559


In [12]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/ID_Migration/By_Division'

In [13]:
print(datetime.datetime.now())
df_2019Q1_sales_all.to_csv("./BL_2019Q1_rewards_and_non_rewards_sales_trans_store_type_"+str(datetime.datetime.now().date())+".csv",index=False)
del df_2019Q1_sales_all
gc.collect()

2019-09-04 12:37:41.097261


310

In [14]:
df_2019Q1_sales_div['store_type'].unique()

array(['SOTF', 'Legacy', 'Ecommerce'], dtype=object)

In [17]:
df_2019Q1_sales_div.head(2)

,location_id,store_type,transaction_dt,rewards_label,division_id,sales,units,trans
0,1,SOTF,2019-02-03,Non_Rewards,1,966.09,605,105
1,1,SOTF,2019-02-03,Non_Rewards,10,3.00,2,1


In [18]:
df_2019Q1_sales_div_agg=df_2019Q1_sales_div.groupby(['rewards_label','store_type','division_id'])['sales','units','trans'].sum().reset_index()
df_2019Q1_sales_div_agg=pd.merge(df_2019Q1_sales_div_agg,divison_name,on="division_id",how="left")
df_2019Q1_sales_div_agg.shape

(50, 7)

In [22]:
df_2019Q1_sales_div_agg.to_csv("./BL_2019Q1_Division_NonRewards_and_Rewards_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

In [ ]:
Stopped and finished here

In [36]:
# 1. by by division, by id group

output_division_with_id=df_2019Q1_sales_div_agg.groupby(['customer_id_hashed','division_id'])['sales','units','trans'].sum().reset_index()

output_division_with_id=pd.merge(output_division_with_id,ID_group_2019Q1_shopped,on="customer_id_hashed",how="outer")

print(output_division_with_id.shape)
output_division_with_id.head(4)


(23846961, 10)


,customer_id_hashed,division_id,sales,units,trans,Group_before_the_Quarter,Group_after_the_Quarter,sales_recent_Quarter,Recent_Quarter_Shopping_Group,NewRewards
0,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...,1,8.81,11,3,Active,Active,31.59,Recent_Quarter_Shopped,Old_Rewards_Members
1,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...,2,14.38,10,3,Active,Active,31.59,Recent_Quarter_Shopped,Old_Rewards_Members
2,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...,6,8.40,3,1,Active,Active,31.59,Recent_Quarter_Shopped,Old_Rewards_Members
3,000007eccbfdc528a745aea90d34404b4fae9fbad2525f...,1,2.79,2,1,NotAvailable_Before_Quarter,Active,15.44,Recent_Quarter_Shopped,Recent_Quarter_New_Sign_Ups


In [37]:
output_division_with_id['customer_id_hashed'].nunique()

8392050

In [38]:
output_division_with_id_nunique=output_division_with_id.groupby(['Group_before_the_Quarter','Group_after_the_Quarter','Recent_Quarter_Shopping_Group','NewRewards','division_id'])['customer_id_hashed'].count().reset_index()

output_division_with_id_sum=output_division_with_id.groupby(['Group_before_the_Quarter','Group_after_the_Quarter','Recent_Quarter_Shopping_Group','NewRewards','division_id'])['sales','units','trans'].sum().reset_index()

output_division_with_id=pd.merge(output_division_with_id_sum,output_division_with_id_nunique,on=['Group_before_the_Quarter','Group_after_the_Quarter','Recent_Quarter_Shopping_Group','NewRewards','division_id'],how="outer")
print(output_division_with_id.shape)
output_division_with_id.head(5)

(52, 9)


,Group_before_the_Quarter,Group_after_the_Quarter,Recent_Quarter_Shopping_Group,NewRewards,division_id,sales,units,trans,customer_id_hashed
0,Active,Active,Recent_Quarter_Shopped,Old_Rewards_Members,1,9.873832e+07,50234361,9821063,4677595
1,Active,Active,Recent_Quarter_Shopped,Old_Rewards_Members,10,1.714465e+05,587518,452241,284875
2,Active,Active,Recent_Quarter_Shopped,Old_Rewards_Members,2,1.024063e+08,30493135,8192600,4270441
3,Active,Active,Recent_Quarter_Shopped,Old_Rewards_Members,3,8.560242e+07,6128308,2578312,1833239
4,Active,Active,Recent_Quarter_Shopped,Old_Rewards_Members,4,9.858662e+07,10937798,4348621,2829381


In [39]:
# 2. by by store types, by id group

output_storetype_with_id=df_2019Q1_sales_div_agg.groupby(['customer_id_hashed','store_type'])['sales','units','trans'].sum().reset_index()

output_storetype_with_id=pd.merge(output_storetype_with_id,ID_group_2019Q1_shopped,on="customer_id_hashed",how="outer")

print(output_storetype_with_id.shape)
output_storetype_with_id.head(4)


(8544804, 10)


,customer_id_hashed,store_type,sales,units,trans,Group_before_the_Quarter,Group_after_the_Quarter,sales_recent_Quarter,Recent_Quarter_Shopping_Group,NewRewards
0,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...,Legacy,31.59,24,7,Active,Active,31.59,Recent_Quarter_Shopped,Old_Rewards_Members
1,000007eccbfdc528a745aea90d34404b4fae9fbad2525f...,Legacy,15.44,9,3,NotAvailable_Before_Quarter,Active,15.44,Recent_Quarter_Shopped,Recent_Quarter_New_Sign_Ups
2,000008374518306e87838f387eedc47d49a447707dc6da...,Legacy,49.77,7,3,Active,Active,49.77,Recent_Quarter_Shopped,Old_Rewards_Members
3,0000095241ba6a94ec0d4b481d2fafd22598cf8cad3399...,Legacy,875.50,6,2,NotAvailable_Before_Quarter,Active,875.50,Recent_Quarter_Shopped,Old_Rewards_Members


In [40]:
output_storetype_with_id_nunique=output_storetype_with_id.groupby(['Group_before_the_Quarter','Group_after_the_Quarter','Recent_Quarter_Shopping_Group','NewRewards','store_type'])['customer_id_hashed'].count().reset_index()

output_storetype_with_id_sum=output_storetype_with_id.groupby(['Group_before_the_Quarter','Group_after_the_Quarter','Recent_Quarter_Shopping_Group','NewRewards','store_type'])['sales','units','trans'].sum().reset_index()

output_storetype_with_id=pd.merge(output_storetype_with_id_sum,output_storetype_with_id_nunique,on=['Group_before_the_Quarter','Group_after_the_Quarter','Recent_Quarter_Shopping_Group','NewRewards','store_type'],how="outer")
print(output_storetype_with_id.shape)
output_storetype_with_id.head(5)

(18, 9)


,Group_before_the_Quarter,Group_after_the_Quarter,Recent_Quarter_Shopping_Group,NewRewards,store_type,sales,units,trans,customer_id_hashed
0,Active,Active,Recent_Quarter_Shopped,Old_Rewards_Members,Ecommerce,5.099926e+06,132598,49604,29496
1,Active,Active,Recent_Quarter_Shopped,Old_Rewards_Members,Legacy,4.905298e+08,94184333,27229480,5454309
2,Active,Active,Recent_Quarter_Shopped,Old_Rewards_Members,SOTF,8.905169e+07,15767259,4564258,973773
3,Active,Active,Recent_Quarter_Shopped,Recent_Quarter_New_Sign_Ups,Ecommerce,1.499700e+02,4,1,1
4,Active,Active,Recent_Quarter_Shopped,Recent_Quarter_New_Sign_Ups,Legacy,2.776962e+04,4007,1101,152


In [41]:
# 3. by by store types, by division, by id group

output_storetype_division_with_id=df_2019Q1_sales_div_agg.groupby(['customer_id_hashed','store_type','division_id'])['sales','units','trans'].sum().reset_index()

output_storetype_division_with_id=pd.merge(output_storetype_division_with_id,ID_group_2019Q1_shopped,on="customer_id_hashed",how="outer")

print(output_storetype_division_with_id.shape)
output_storetype_division_with_id.head(4)


(24077194, 11)


,customer_id_hashed,store_type,division_id,sales,units,trans,Group_before_the_Quarter,Group_after_the_Quarter,sales_recent_Quarter,Recent_Quarter_Shopping_Group,NewRewards
0,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...,Legacy,1,8.81,11,3,Active,Active,31.59,Recent_Quarter_Shopped,Old_Rewards_Members
1,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...,Legacy,2,14.38,10,3,Active,Active,31.59,Recent_Quarter_Shopped,Old_Rewards_Members
2,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...,Legacy,6,8.40,3,1,Active,Active,31.59,Recent_Quarter_Shopped,Old_Rewards_Members
3,000007eccbfdc528a745aea90d34404b4fae9fbad2525f...,Legacy,1,2.79,2,1,NotAvailable_Before_Quarter,Active,15.44,Recent_Quarter_Shopped,Recent_Quarter_New_Sign_Ups


In [42]:
output_storetype_division_with_id_nunique=output_storetype_division_with_id.groupby(['Group_before_the_Quarter','Group_after_the_Quarter','Recent_Quarter_Shopping_Group','NewRewards','store_type','division_id'])['customer_id_hashed'].count().reset_index()

output_storetype_division_with_id_sum=output_storetype_division_with_id.groupby(['Group_before_the_Quarter','Group_after_the_Quarter','Recent_Quarter_Shopping_Group','NewRewards','store_type','division_id'])['sales','units','trans'].sum().reset_index()

output_storetype_division=pd.merge(output_storetype_division_with_id_sum,output_storetype_division_with_id_nunique,on=['Group_before_the_Quarter','Group_after_the_Quarter','Recent_Quarter_Shopping_Group','NewRewards','store_type','division_id'],how="outer")
print(output_storetype_division.shape)
output_storetype_division.head(5)

(133, 10)


,Group_before_the_Quarter,Group_after_the_Quarter,Recent_Quarter_Shopping_Group,NewRewards,store_type,division_id,sales,units,trans,customer_id_hashed
0,Active,Active,Recent_Quarter_Shopped,Old_Rewards_Members,Ecommerce,1,114250.17,3642,1467,1299
1,Active,Active,Recent_Quarter_Shopped,Old_Rewards_Members,Ecommerce,2,15235.35,1322,525,474
2,Active,Active,Recent_Quarter_Shopped,Old_Rewards_Members,Ecommerce,3,2576205.59,45295,19707,13982
3,Active,Active,Recent_Quarter_Shopped,Old_Rewards_Members,Ecommerce,4,827334.64,56308,16443,12445
4,Active,Active,Recent_Quarter_Shopped,Old_Rewards_Members,Ecommerce,5,158843.05,12644,3179,2767


In [43]:
output_storetype_division.shape

(133, 10)

In [44]:
#### 
writer=pd.ExcelWriter("./BL_2019Q1_rewards_summary_div_store_type_groups_data_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
output_division_with_id.to_excel(writer,"output_division",index=False)
output_storetype_with_id.to_excel(writer,"output_store",index=False)
output_storetype_division.to_excel(writer,"output_storetype_division",index=False)
writer.save()